In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
import pickle
import json
from pathlib import Path
import time

# ============================================================================
# PAGE CONFIG
# ============================================================================

st.set_page_config(
    page_title="Halal/Haram Food Classifier",
    page_icon="🍔",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ============================================================================
# CUSTOM CSS
# ============================================================================

st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #2E86AB;
        text-align: center;
        font-weight: bold;
        margin-bottom: 1rem;
    }
    .sub-header {
        font-size: 1.2rem;
        color: #666;
        text-align: center;
        margin-bottom: 2rem;
    }
    .halal-result {
        background: linear-gradient(135deg, #06A77D 0%, #05d69e 100%);
        padding: 2rem;
        border-radius: 15px;
        text-align: center;
        color: white;
        font-size: 2rem;
        font-weight: bold;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        margin: 1rem 0;
    }
    .haram-result {
        background: linear-gradient(135deg, #C73E1D 0%, #e74c3c 100%);
        padding: 2rem;
        border-radius: 15px;
        text-align: center;
        color: white;
        font-size: 2rem;
        font-weight: bold;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        margin: 1rem 0;
    }
    .confidence-box {
        background: #f8f9fa;
        padding: 1rem;
        border-radius: 10px;
        margin: 1rem 0;
    }
    .ingredient-box {
        background: #e9ecef;
        padding: 1rem;
        border-radius: 10px;
        margin: 0.5rem 0;
        font-family: monospace;
    }
    .stButton>button {
        width: 100%;
        background: linear-gradient(135deg, #2E86AB 0%, #1a5f7a 100%);
        color: white;
        font-size: 1.2rem;
        padding: 0.8rem;
        border: none;
        border-radius: 10px;
        font-weight: bold;
    }
    .stButton>button:hover {
        background: linear-gradient(135deg, #1a5f7a 0%, #0d3b4d 100%);
    }
</style>
""", unsafe_allow_html=True)

# ============================================================================
# LOAD MODEL AND TOKENIZER
# ============================================================================

@st.cache_resource
def load_model(model_path):
    """Load Keras or TFLite model"""
    try:
        if model_path.endswith('.h5'):
            model = tf.keras.models.load_model(model_path)
            st.success(f"✅ Loaded Keras model: {model_path}")
            return model, 'keras'
        elif model_path.endswith('.tflite'):
            interpreter = tf.lite.Interpreter(model_path=model_path)
            interpreter.allocate_tensors()
            st.success(f"✅ Loaded TFLite model: {model_path}")
            return interpreter, 'tflite'
    except Exception as e:
        st.error(f"❌ Error loading model: {e}")
        return None, None

@st.cache_resource
def load_tokenizer(tokenizer_path):
    """Load tokenizer from pickle or JSON"""
    try:
        if tokenizer_path.endswith('.pkl'):
            with open(tokenizer_path, 'rb') as f:
                tokenizer = pickle.load(f)
            st.success(f"✅ Loaded tokenizer from pickle")
            return tokenizer, 'pickle'
        elif tokenizer_path.endswith('.json'):
            with open(tokenizer_path, 'r', encoding='utf-8') as f:
                tokenizer_json = json.load(f)
            st.success(f"✅ Loaded tokenizer from JSON")
            return tokenizer_json, 'json'
    except Exception as e:
        st.error(f"❌ Error loading tokenizer: {e}")
        return None, None

# ============================================================================
# TEXT PREPROCESSING
# ============================================================================

def preprocess_text(text, tokenizer, tokenizer_type, max_length=256):
    """Preprocess text for model input"""
    
    if tokenizer_type == 'pickle':
        # Keras tokenizer
        sequences = tokenizer.texts_to_sequences([text])
        padded = tf.keras.preprocessing.sequence.pad_sequences(
            sequences, 
            maxlen=max_length, 
            padding='post', 
            truncating='post'
        )
        return padded
    
    elif tokenizer_type == 'json':
        # Manual tokenization from JSON
        word_index = tokenizer['word_index']
        config = tokenizer['config']
        
        # Lowercase if needed
        if config.get('lower', True):
            text = text.lower()
        
        # Split
        words = text.split(config.get('split', ' '))
        
        # Map to indices
        sequence = []
        for word in words:
            if word in word_index:
                sequence.append(word_index[word])
            elif config.get('oov_token') and config['oov_token'] in word_index:
                sequence.append(word_index[config['oov_token']])
        
        # Pad
        if len(sequence) < max_length:
            sequence = sequence + [0] * (max_length - len(sequence))
        else:
            sequence = sequence[:max_length]
        
        return np.array([sequence], dtype=np.int32)

# ============================================================================
# PREDICTION
# ============================================================================

def predict(text, model, model_type, tokenizer, tokenizer_type, max_length=256):
    """Make prediction"""
    
    # Preprocess
    input_data = preprocess_text(text, tokenizer, tokenizer_type, max_length)
    
    # Predict
    if model_type == 'keras':
        prediction = model.predict(input_data, verbose=0)
        probability = float(prediction[0][0])
    
    elif model_type == 'tflite':
        # Get input/output details
        input_details = model.get_input_details()
        output_details = model.get_output_details()
        
        # Set input
        model.set_tensor(input_details[0]['index'], input_data)
        
        # Run inference
        model.invoke()
        
        # Get output
        output = model.get_tensor(output_details[0]['index'])
        probability = float(output[0][0])
    
    # Classify
    is_haram = probability > 0.5
    confidence = probability if is_haram else (1 - probability)
    
    return {
        'is_haram': is_haram,
        'probability': probability,
        'confidence': confidence,
        'label': 'HARAM' if is_haram else 'HALAL'
    }

# ============================================================================
# MAIN APP
# ============================================================================

def main():
    # Header
    st.markdown('<div class="main-header">🍔 Halal/Haram Food Classifier</div>', unsafe_allow_html=True)
    st.markdown('<div class="sub-header">Bilingual Deep Learning Model for Food Ingredient Classification</div>', unsafe_allow_html=True)
    
    # Sidebar
    with st.sidebar:
        st.header("⚙️ Configuration")
        
        # Model selection
        st.subheader("📦 Model")
        model_files = list(Path('models').glob('*.h5')) + list(Path('models').glob('*.tflite'))
        
        if not model_files:
            st.error("❌ No models found in 'models/' directory!")
            st.info("Please add your .h5 or .tflite model files to the 'models/' folder")
            return
        
        model_file = st.selectbox(
            "Select Model",
            model_files,
            format_func=lambda x: x.name
        )
        
        # Tokenizer selection
        st.subheader("📝 Tokenizer")
        tokenizer_files = list(Path('models').glob('*.pkl')) + list(Path('models').glob('tokenizer*.json'))
        
        if not tokenizer_files:
            st.error("❌ No tokenizer found!")
            st.info("Please add tokenizer.pkl or tokenizer.json to the 'models/' folder")
            return
        
        tokenizer_file = st.selectbox(
            "Select Tokenizer",
            tokenizer_files,
            format_func=lambda x: x.name
        )
        
        # Max length
        max_length = st.slider("Max Sequence Length", 64, 512, 256, 64)
        
        # Load button
        if st.button("🔄 Load Model & Tokenizer"):
            st.session_state.model, st.session_state.model_type = load_model(str(model_file))
            st.session_state.tokenizer, st.session_state.tokenizer_type = load_tokenizer(str(tokenizer_file))
            st.session_state.max_length = max_length
        
        st.markdown("---")
        
        # About
        st.subheader("ℹ️ About")
        st.info("""
        **How it works:**
        1. Enter food ingredients
        2. Model analyzes text
        3. Get halal/haram prediction
        
        **Supports:**
        - English & Indonesian
        - Multiple ingredients
        - Real-time prediction
        """)
        
        # Stats
        if 'prediction_count' in st.session_state:
            st.metric("Predictions Made", st.session_state.prediction_count)
    
    # Main content
    if 'model' not in st.session_state or st.session_state.model is None:
        st.warning("⚠️ Please load a model from the sidebar first!")
        
        st.markdown("### 📚 Quick Start Guide")
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.markdown("#### 1️⃣ Prepare Files")
            st.code("""
models/
  ├── your_model.h5
  └── tokenizer.pkl
            """)
        
        with col2:
            st.markdown("#### 2️⃣ Load Model")
            st.write("Select model and tokenizer from sidebar")
        
        with col3:
            st.markdown("#### 3️⃣ Start Testing")
            st.write("Enter ingredients and get predictions!")
        
        return
    
    # Input methods
    st.header("📝 Input Ingredients")
    
    tab1, tab2, tab3 = st.tabs(["✍️ Manual Input", "📄 Examples", "🔢 Batch Testing"])
    
    with tab1:
        st.subheader("Enter Food Ingredients")
        
        # Text input
        user_input = st.text_area(
            "Ingredients (comma-separated):",
            placeholder="Example: chicken, salt, pepper, onion, garlic\nContoh: ayam, garam, lada, bawang, bawang putih",
            height=150,
            help="Enter ingredients in English or Indonesian, separated by commas"
        )
        
        # Language hint
        col1, col2 = st.columns(2)
        with col1:
            st.caption("🇬🇧 English: chicken, beef, pork, wine, gelatin")
        with col2:
            st.caption("🇮🇩 Indonesian: ayam, sapi, babi, arak, gelatin")
        
        # Predict button
        if st.button("🔮 Predict", type="primary"):
            if user_input.strip():
                with st.spinner("Analyzing ingredients..."):
                    # Predict
                    result = predict(
                        user_input,
                        st.session_state.model,
                        st.session_state.model_type,
                        st.session_state.tokenizer,
                        st.session_state.tokenizer_type,
                        st.session_state.max_length
                    )
                    
                    # Update counter
                    if 'prediction_count' not in st.session_state:
                        st.session_state.prediction_count = 0
                    st.session_state.prediction_count += 1
                    
                    # Display results
                    st.markdown("---")
                    st.subheader("🎯 Prediction Results")
                    
                    # Result box
                    if result['is_haram']:
                        st.markdown(f'<div class="haram-result">⚠️ HARAM</div>', unsafe_allow_html=True)
                    else:
                        st.markdown(f'<div class="halal-result">✅ HALAL</div>', unsafe_allow_html=True)
                    
                    # Details
                    col1, col2, col3 = st.columns(3)
                    
                    with col1:
                        st.metric("Confidence", f"{result['confidence']*100:.2f}%")
                    
                    with col2:
                        st.metric("Probability Score", f"{result['probability']:.4f}")
                    
                    with col3:
                        status = "🔴 High Risk" if result['is_haram'] else "🟢 Safe"
                        st.metric("Status", status)
                    
                    # Ingredients display
                    st.markdown("### 🧾 Analyzed Ingredients:")
                    ingredients = [i.strip() for i in user_input.split(',')]
                    
                    cols = st.columns(3)
                    for idx, ingredient in enumerate(ingredients):
                        with cols[idx % 3]:
                            st.markdown(f'<div class="ingredient-box">• {ingredient}</div>', unsafe_allow_html=True)
                    
                    # Interpretation
                    st.markdown("### 💡 Interpretation")
                    if result['is_haram']:
                        st.error("""
                        **⚠️ This product may contain haram ingredients.**
                        
                        The model detected patterns associated with non-halal ingredients. 
                        Please verify the ingredient list carefully before consumption.
                        """)
                    else:
                        st.success("""
                        **✅ This product appears to be halal.**
                        
                        The model did not detect any haram ingredients. However, 
                        always check for halal certification for complete assurance.
                        """)
            else:
                st.warning("⚠️ Please enter some ingredients first!")
    
    with tab2:
        st.subheader("📄 Example Ingredients")
        
        examples = {
            "✅ Halal Example 1": "chicken breast, salt, black pepper, olive oil, garlic",
            "✅ Halal Example 2": "ayam, garam, lada hitam, minyak zaitun, bawang putih",
            "✅ Halal Example 3": "beef, potato, carrot, onion, tomato sauce",
            "⚠️ Haram Example 1": "pork sausage, bacon, ham, lard",
            "⚠️ Haram Example 2": "wine, beer, alcohol, rum extract",
            "⚠️ Haram Example 3": "gelatin (pork), bacon bits, lard, pork fat",
        }
        
        for name, ingredients in examples.items():
            with st.expander(name):
                st.code(ingredients)
                if st.button(f"Test: {name}", key=name):
                    # Auto-fill and trigger prediction
                    st.session_state.example_input = ingredients
                    st.rerun()
        
        # Auto-fill from example
        if 'example_input' in st.session_state:
            st.info(f"Example loaded: {st.session_state.example_input}")
            if st.button("🔮 Predict Example"):
                result = predict(
                    st.session_state.example_input,
                    st.session_state.model,
                    st.session_state.model_type,
                    st.session_state.tokenizer,
                    st.session_state.tokenizer_type,
                    st.session_state.max_length
                )
                
                if result['is_haram']:
                    st.markdown(f'<div class="haram-result">⚠️ HARAM</div>', unsafe_allow_html=True)
                else:
                    st.markdown(f'<div class="halal-result">✅ HALAL</div>', unsafe_allow_html=True)
                
                st.metric("Confidence", f"{result['confidence']*100:.2f}%")
                
                del st.session_state.example_input
    
    with tab3:
        st.subheader("🔢 Batch Testing")
        
        st.info("Upload a CSV file with 'ingredients' column for batch prediction")
        
        uploaded_file = st.file_uploader("Upload CSV", type=['csv'])
        
        if uploaded_file:
            import pandas as pd
            
            df = pd.read_csv(uploaded_file)
            st.write("Preview:", df.head())
            
            if 'ingredients' in df.columns:
                if st.button("🚀 Run Batch Prediction"):
                    with st.spinner("Processing batch..."):
                        results = []
                        progress_bar = st.progress(0)
                        
                        for idx, row in df.iterrows():
                            result = predict(
                                row['ingredients'],
                                st.session_state.model,
                                st.session_state.model_type,
                                st.session_state.tokenizer,
                                st.session_state.tokenizer_type,
                                st.session_state.max_length
                            )
                            results.append(result)
                            progress_bar.progress((idx + 1) / len(df))
                        
                        df['prediction'] = [r['label'] for r in results]
                        df['confidence'] = [r['confidence'] for r in results]
                        
                        st.success("✅ Batch prediction complete!")
                        st.dataframe(df)
                        
                        # Download results
                        csv = df.to_csv(index=False).encode('utf-8')
                        st.download_button(
                            "📥 Download Results",
                            csv,
                            "batch_predictions.csv",
                            "text/csv"
                        )
            else:
                st.error("CSV must have 'ingredients' column!")

if __name__ == "__main__":
    main()